In [19]:
import random
from dimod.sampleset import SampleSet
from dwave.system.composites.embedding import EmbeddedStructure
import networkx as nx
import pandas as pd
import sys
from collections import defaultdict
from dwave.system import DWaveSampler, EmbeddingComposite
import plotly.express as px
import plotly.graph_objects as go

In [40]:
random.seed(10)
#n = 3
n_j, n_y, n_z = 4, 3, 2
G = nx.grid_graph(dim=(n_z,n_y,n_x))
# G_edges = []
# nodes = []
# for i in range(n):
#     for j in range(n):
#         for k in range(n):
#             nodes.append((i,j,k))
# enumerate(nodes
# for index, node in enumerate(nodes):
#     for index_2, node_2 in enumerate(nodes):
#         if index < index_2:
#             if sum([node[i] - node_2[i] for i in range(3)])==1:
#                 G_edges.append((index, index_2))
# print(G_edges)
# print(nodes)
# exit()

In [52]:
h = {}
J = defaultdict(int)

# Objective
for i, j in G.edges:
    random_val = 2*random.randint(0, 1) - 1
    J[(i, j)] = random_val
    #Q[(j, i)] = random_val

print(J)
sampler = EmbeddingComposite(DWaveSampler())

defaultdict(<class 'int'>, {((0, 0, 0), (1, 0, 0)): -1, ((0, 0, 0), (0, 1, 0)): -1, ((0, 0, 0), (0, 0, 1)): 1, ((0, 0, 1), (1, 0, 1)): -1, ((0, 0, 1), (0, 1, 1)): -1, ((0, 1, 0), (1, 1, 0)): 1, ((0, 1, 0), (0, 2, 0)): 1, ((0, 1, 0), (0, 1, 1)): -1, ((0, 1, 1), (1, 1, 1)): 1, ((0, 1, 1), (0, 2, 1)): -1, ((0, 2, 0), (1, 2, 0)): -1, ((0, 2, 0), (0, 2, 1)): -1, ((0, 2, 1), (1, 2, 1)): 1, ((1, 0, 0), (2, 0, 0)): -1, ((1, 0, 0), (1, 1, 0)): 1, ((1, 0, 0), (1, 0, 1)): -1, ((1, 0, 1), (2, 0, 1)): 1, ((1, 0, 1), (1, 1, 1)): 1, ((1, 1, 0), (2, 1, 0)): -1, ((1, 1, 0), (1, 2, 0)): 1, ((1, 1, 0), (1, 1, 1)): -1, ((1, 1, 1), (2, 1, 1)): -1, ((1, 1, 1), (1, 2, 1)): -1, ((1, 2, 0), (2, 2, 0)): 1, ((1, 2, 0), (1, 2, 1)): -1, ((1, 2, 1), (2, 2, 1)): -1, ((2, 0, 0), (3, 0, 0)): 1, ((2, 0, 0), (2, 1, 0)): 1, ((2, 0, 0), (2, 0, 1)): 1, ((2, 0, 1), (3, 0, 1)): 1, ((2, 0, 1), (2, 1, 1)): -1, ((2, 1, 0), (3, 1, 0)): 1, ((2, 1, 0), (2, 2, 0)): 1, ((2, 1, 0), (2, 1, 1)): -1, ((2, 1, 1), (3, 1, 1)): -1, ((2, 1, 

In [51]:

sampleset = sampler.sample_ising(h, J, num_reads=10)

In [42]:
df = sampleset.to_pandas_dataframe()
df.to_csv("output")

In [43]:
print(df)

   0                 1           ...  2  3                 \
   0     1     2     0     1     ...  2  0     1     2      
   0  1  0  1  0  1  0  1  0  1  ...  1  0  1  0  1  0  1   
0 -1 -1  1 -1 -1  1 -1 -1  1 -1  ... -1  1  1  1 -1 -1  1   
1 -1  1  1  1 -1  1  1  1 -1  1  ... -1 -1 -1 -1  1  1  1   
2  1  1 -1  1  1 -1  1  1 -1  1  ...  1 -1 -1 -1  1  1 -1   
3 -1  1  1  1 -1  1 -1 -1  1 -1  ... -1  1  1  1 -1 -1  1   
4  1  1 -1  1  1 -1  1  1 -1  1  ...  1 -1  1 -1 -1  1 -1   

  chain_break_fraction energy num_occurrences  
                                               
                                               
0                  0.0  -30.0               1  
1                  0.0  -30.0               3  
2                  0.0  -30.0               4  
3                  0.0  -28.0               1  
4                  0.0  -28.0               1  

[5 rows x 27 columns]


In [44]:
x, y, z, spin = [],[],[],[]
for i in range(n_x):
    for j in range(n_y):
        for k in range(n_z):
            x.append(i)
            y.append(j)
            z.append(k)
            spin.append(df[(i,j,k)][0])
df_3d_model = pd.DataFrame({"x": x, "y":y, "z":z, "spin":spin})

In [46]:
fig = px.scatter_3d(data_frame=df_3d_model,x="x", y="y", z="z", color="spin")
color_map={1: 'rgba(255,0,0,1)', -1:'rgba(0,255,0,1)'}
for edge in J:
    df_line = pd.DataFrame({"x": [edge[0][0], edge[1][0]], "y": [edge[0][1], edge[1][1]], "z": [edge[0][2], edge[1][2]], "align":[J[edge], J[edge]]})
    new_fig = px.line_3d(df_line, x="x", y="y", z="z", color="align", color_discrete_map=color_map)
    fig = go.Figure(data=fig.data + new_fig.data)
fig.show()